# Mein erstes TensorFlow Expermient: MNIST set
Folgendes Tutorial: https://www.tensorflow.org/tutorials/quickstart/beginner

In [1]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)

TensorFlow version: 2.1.0


In [2]:
import keras

Using TensorFlow backend.


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 1s 0us/step


The 'x' is image (pixel array) and 'y' is the label (number).

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [5]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[-0.4740513 ,  0.52910966,  0.38059938,  0.62839496, -0.08999389,
         0.02807391, -0.17965865, -0.24809535,  0.0950073 , -0.28039572]],
      dtype=float32)

Take the first image (as a array \[:1\] and not as a single element \[0\]) and ask the model for the prediction.

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.05622546, 0.15332054, 0.13216095, 0.16932434, 0.0825519 ,
        0.09289734, 0.07547206, 0.07047977, 0.09932809, 0.06823962]],
      dtype=float32)

This is the prediction as a percentage (the sum of them all is 1). The prediction is quire random, because the model has not been trained yet.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.3762603

The difference bettween the actual value (label, y) and the predicted value (x) is the loss, which will now be reduced/optimized by training the model.

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 68us/sample - loss: 0.2943 - accuracy: 0.9142
Epoch 2/5
60000/60000 [==============================] - 4s 61us/sample - loss: 0.1444 - accuracy: 0.9563
Epoch 3/5
60000/60000 [==============================] - 4s 61us/sample - loss: 0.1080 - accuracy: 0.9673
Epoch 4/5
60000/60000 [==============================] - 4s 59us/sample - loss: 0.0874 - accuracy: 0.9728
Epoch 5/5
60000/60000 [==============================] - 4s 60us/sample - loss: 0.0758 - accuracy: 0.9756


In [11]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0732 - accuracy: 0.9778


[0.07315649647619575, 0.9778]

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

How we can explore our model. We take the first three images, find the index of the highest value (the one with the smallest negative exponent) and that gives us the predicted number.

In [13]:
probability_model(x_test[:3])

<tf.Tensor: shape=(3, 10), dtype=float32, numpy=
array([[7.4850615e-09, 1.2581109e-08, 3.6912365e-06, 2.7360325e-04,
        1.1394843e-12, 1.1169980e-07, 2.3571528e-13, 9.9971884e-01,
        4.0544691e-07, 3.3391111e-06],
       [1.3067257e-07, 5.0347142e-05, 9.9994814e-01, 1.0975275e-06,
        1.0323277e-15, 9.3503871e-08, 4.0239531e-08, 3.4105254e-13,
        2.1070679e-07, 3.1929690e-15],
       [5.2496182e-07, 9.9789232e-01, 1.8723743e-04, 2.6879172e-05,
        2.0412512e-05, 1.1285584e-05, 4.3190954e-05, 1.6639493e-03,
        1.4978237e-04, 4.3929035e-06]], dtype=float32)>

Wen can then verify with the actual numbers from the label set (y).

In [14]:
y_test[:3]

array([7, 2, 1], dtype=uint8)